In [150]:
####階層ベイズプロビットモデル####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [161]:
####データの発生####
#np.random.seed(8742)   #シードを設定

##データの設定
hh = 2000   #ユーザー数
k = 7   #パラメータ数
n = poisson(numpy.random.gamma(40, 1/1.5, hh), hh)   #1人あたりの観測数
N = np.sum(n)

In [162]:
##IDの設定
u_list = [i for i in range(hh)]
t_list = [i for i in range(hh)]
u_id = np.zeros(N, dtype='int')
t_id = np.zeros(N, dtype='int')
start = 0

for i in range(hh):
    #リストに格納
    u_list[i] = np.repeat(i, n[i])
    t_list[i] = np.array(range(n[i]))+1
    
    #リストをベクトル変換
    u_id[start:start+n[i]] = u_list[i]
    t_id[start:start+n[i]] = t_list[i]
    start += n[i] 

In [185]:
##説明変数の発生
intercept = np.transpose(np.matlib.repmat(1, 1, N))
Z1 = np.log(numpy.random.gamma(5, 1, N)) * numpy.random.binomial(1, 0.25, N)   #降水量の対数
Z2 = numpy.random.uniform(0, 1, N)   #チラシ商品の平均値引率
Z3 = np.log(numpy.random.poisson(numpy.random.gamma(25, 1/0.5, N)))   #チラシ掲載商品数の対数
Z3 = Z3 - np.min(Z3)
Z4 = numpy.random.binomial(1, 0.5, N)   #他店チラシ状況
Z = np.concatenate((intercept, Z1[:, np.newaxis], Z2[:, np.newaxis], Z3[:, np.newaxis], Z4[:, np.newaxis]), axis=1)   #データの結合

In [186]:
##パラメータの設定
beta0 = numpy.random.normal(-0.4, 0.4, hh)
beta1 = numpy.random.normal(-0.5, 0.3, hh)
beta2 = numpy.random.normal(0.5, 0.25, hh)
beta3 = numpy.random.normal(0.3, 0.25, hh)
beta4 = numpy.random.normal(-0.6, 0.2, hh)
beta = np.concatenate((beta0[:, np.newaxis], beta1[:, np.newaxis], beta2[:, np.newaxis], beta3[:, np.newaxis],
                       beta4[:, np.newaxis]), axis=1)   #回帰係数の結合

In [187]:
##応答変数の発生
mu = np.sum(Z * beta[u_id, :], axis=1)   #潜在効用の平均構造
U = numpy.random.normal(mu, 1, N)   #潜在効用を生成
y = (U >= 0)*1 + (U < 0)*0   #購買有無に変換
np.mean(y)

0.42095384673135755

In [202]:
####マルコフ連鎖モンテカルロ法で階層ベイズプロビットモデルを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [223]:
##アルゴリズムの設定
R = 10000
keep = 4
burnin = int(R / 4)
disp = 8
iter = 0
k = Z.shape[1]

5

In [214]:
np.mean(rtnorm(1.5, 1, 0, 100, 100000))

1.6410317562722487